# PYTHON SQLITE BASICS

SQLite comes in built with Python3. No installation is required. 
We will import 3 packages here - sqlite3, pandas and sys

## Import packages

In [15]:
import pandas as pd
import sqlite3
import sys
import os

## Make Sqlite3 connection Object
This sets the location of the database on disk
* If the database is not present, it will create a new database
* If the database is present, it will connect to the database

In [16]:
conn    = sqlite3.connect('../Python_SQLite/data/example.db')

## Create Cursor
Cursor class is an instance which can invoke methods (functions in class) that execute SQLite statements, fetch data from the result sets of the queries.

In [17]:
cursr   = conn.cursor()

## Create Table
We will write a class to create table in Sqlite database which can be called as necessary

In [18]:
class CreateTable:

    def __init__(self, cursr, table_name, table_defn):
        self.cursr          = cursr
        self.table_name     = table_name
        self.table_defn     = table_defn

    def create_table(self):
        try:
            self.cursr.execute(f'''CREATE TABLE {self.table_name} {tuple(self.table_defn)}''')
            print ('Table Created Sucessfully - {} '.format(self.table_name))
        except :
            print('Table Creation Failed - {}'.format(sys.exc_info()[0]))
        return

Next we will create the definition. We  will need the following to define the table
* Table Name
* Table Column Names
* Table Column DataType
* Constrains

In [19]:
table_name = 'stocks'
table_defn = {  'ID'       : 'real primary key', 
                'DATE'     : 'text',
                'TRANS'    : 'text', 
                'SYMBOL'   : 'text', 
                'QTY'      : 'real',
                'PRICE'    : 'real'
             }

Create Table

In [20]:
create_table = CreateTable(cursr, table_name, table_defn)
create_table.create_table()

Table Created Sucessfully - stocks 


Nice ! Table is created . Lets insert some data into the table

## Insert Records Into Table

In [21]:
class InsertIntoTable:

    def __init__(self, conn, cursr, table_name):
        self.conn           = conn
        self.cursr          = cursr
        self.table_name     = table_name

    def insert_record(self, record):
        try:
            self.record = record
            self.cursr.execute(f'''INSERT INTO {self.table_name} VALUES {self.record}''')
            self.conn.commit()
            print ('Record Updated Sucessfully - {} '.format(self.record))
        except :
            print ('Record Update Failed - {}'.format(sys.exc_info()[0]))
        return

Let's Update some records

In [22]:
update_rec_into_table = InsertIntoTable(conn, cursr, table_name)
update_rec_into_table.insert_record((111, '2006-01-05','BUY', 'RHEL',100,35.14))
update_rec_into_table.insert_record((222, '2006-01-06','SELL','BABA',105,20.14))
update_rec_into_table.insert_record((333, '2006-01-05','BUY', 'MSFT',100,40.21))
update_rec_into_table.insert_record((444, '2006-01-06','SELL','TSLA',105,60.45))

Record Updated Sucessfully - (111, '2006-01-05', 'BUY', 'RHEL', 100, 35.14) 
Record Updated Sucessfully - (222, '2006-01-06', 'SELL', 'BABA', 105, 20.14) 
Record Updated Sucessfully - (333, '2006-01-05', 'BUY', 'MSFT', 100, 40.21) 
Record Updated Sucessfully - (444, '2006-01-06', 'SELL', 'TSLA', 105, 60.45) 


## Fetch Data from Table

Table is now updated with data. Let's fetch the data from table

In [23]:
class FetchFromTable:

    def __init__(self, conn):
        self.conn = conn
    
    def fetch_data(self, query):
        self.query = query        
        query_result_df = pd.read_sql(query, self.conn)
        return query_result_df

In [24]:
#Fetch data
query = f'''SELECT * FROM {table_name}'''
fetch_data_from_table = FetchFromTable(conn)
qry_result = fetch_data_from_table.fetch_data(query)
qry_result

,ID,DATE,TRANS,SYMBOL,QTY,PRICE
0,111,2006-01-05,BUY,RHEL,100,35.14
1,222,2006-01-06,SELL,BABA,105,20.14
2,333,2006-01-05,BUY,MSFT,100,40.21
3,444,2006-01-06,SELL,TSLA,105,60.45


Congratulations for coming so far. We created the table sucessfully, Updated Record and fetched the data. Lets try another query

In [25]:
#Fetch data
query = f'''SELECT * FROM {table_name} WHERE PRICE BETWEEN 20 AND 40'''
fetch_data_from_table = FetchFromTable(conn)
fetch_data_from_table.fetch_data(query)

,ID,DATE,TRANS,SYMBOL,QTY,PRICE
0,111,2006-01-05,BUY,RHEL,100,35.14
1,222,2006-01-06,SELL,BABA,105,20.14


## Update Record Into Table

In [26]:
class UpdateTable:

    def __init__(self, conn, cursr, table_name, update_query):
        self.cursr          = cursr
        self.table_name     = table_name
        self.update_query   = update_query
        print(update_query)

    def update_record(self):
        try:
            self.cursr.execute(update_query)
            print ('Record Updated Sucessfully!')
        except :
            print ('Record Update Failed - {}'.format(sys.exc_info()[0]))
        return

In [27]:
update_query = f'''UPDATE {table_name} SET SYMBOL='DADA' WHERE ID=222'''
update_tble = UpdateTable(conn, cursr, table_name, update_query)
update_tble.update_record()

UPDATE stocks SET SYMBOL='DADA' WHERE ID=222
Record Updated Sucessfully!


In [28]:
#Fetch data
query = f'''SELECT * FROM {table_name}'''
fetch_data_from_table = FetchFromTable(conn)
qry_result = fetch_data_from_table.fetch_data(query)
qry_result

,ID,DATE,TRANS,SYMBOL,QTY,PRICE
0,111,2006-01-05,BUY,RHEL,100,35.14
1,222,2006-01-06,SELL,DADA,105,20.14
2,333,2006-01-05,BUY,MSFT,100,40.21
3,444,2006-01-06,SELL,TSLA,105,60.45


## List All Tables in Database
This class will list all the tables inside a database

In [29]:
class ListAllTables:

    def __init__(self, conn):
        self.conn = conn
    
    def list_all_tables(self, query):
        self.query = query        
        table_list = pd.read_sql(query, self.conn)
        return table_list

In [30]:
query_list_all_tables = f"""SELECT NAME FROM sqlite_master WHERE TYPE = 'table'"""
list_all_tabl_in_db = ListAllTables(conn)
list_all_tabl_in_db.list_all_tables(query_list_all_tables)

,name
0,stocks


## Drop Table From Database

In [31]:
class DropTable:

    def __init__(self, cursr, table_name):
        self.cursr          = cursr
        self.table_name     = table_name

    def drop_table(self):
        try:
            self.cursr.execute(f'''DROP TABLE {self.table_name}''')
            print ('Table Dropped Sucessfully - {} '.format(self.table_name))
        except :
            print('Table Drop Failed - {}'.format(sys.exc_info()[0]))
        return

In [32]:
droptable = DropTable(cursr, table_name)
droptable.drop_table()

Table Dropped Sucessfully - stocks 
